# Code Generator

The requirement: use a Frontier model to generate high performance C++ code from Python code


In [1]:
# imports

import os
import io
import sys
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import subprocess


In [ ]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

In [ ]:
# Connect to client libraries

openai = OpenAI()


gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"


gemini = OpenAI(api_key=google_api_key, base_url=gemini_url)



In [ ]:
models = ["gpt-5","gemini-2.5-pro", "qwen2.5-coder", "deepseek-coder-v2" ]

clients = {"gpt-5": openai, "gemini-2.5-pro": gemini,"qwen2.5-coder": ollama, "deepseek-coder-v2": ollama,}

# Want to keep costs ultra-low? Replace this with models of your choice, using the examples from yesterday

In [5]:
from system_info import retrieve_system_info

system_info = retrieve_system_info()
system_info

{'os': {'system': 'Darwin',
  'arch': 'arm64',
  'release': '23.1.0',
  'version': 'Darwin Kernel Version 23.1.0: Mon Oct  9 21:28:12 PDT 2023; root:xnu-10002.41.9~6/RELEASE_ARM64_T8103',
  'kernel': '23.1.0',
  'distro': None,
  'wsl': False,
  'rosetta2_translated': False,
  'target_triple': 'arm64-apple-darwin23.1.0'},
 'package_managers': ['xcode-select (CLT)', 'brew'],
 'cpu': {'brand': 'Apple M1',
  'cores_logical': 8,
  'cores_physical': 8,
  'simd': []},
 'toolchain': {'compilers': {'gcc': 'Apple clang version 15.0.0 (clang-1500.0.40.1)',
   'g++': 'Apple clang version 15.0.0 (clang-1500.0.40.1)',
   'clang': 'Apple clang version 15.0.0 (clang-1500.0.40.1)',
   'msvc_cl': ''},
  'build_tools': {'cmake': '', 'ninja': '', 'make': 'GNU Make 3.81'},
  'linkers': {'ld_lld': ''}}}

## Overwrite this with the commands from yesterday

Or just use the website like yesterday:

 https://www.programiz.com/cpp-programming/online-compiler/

In [6]:
compile_command = ["clang++", "-std=c++17", "-Ofast", "-mcpu=native", "-flto=thin", "-fvisibility=hidden", "-DNDEBUG", "main.cpp", "-o", "main"]
run_command = ["./main"]


## And now, on with the main task

In [7]:
system_prompt = """
Your task is to convert Python code into high performance C++ code.
Respond only with C++ code. Do not provide any explanation other than occasional comments.
The C++ response needs to produce an identical output in the fastest possible time.
"""

def user_prompt_for(python):
    return f"""
Port this Python code to C++ with the fastest possible implementation that produces identical output in the least time.
The system information is:
{system_info}
Your response will be written to a file called main.cpp and then compiled and executed; the compilation command is:
{compile_command}
Respond only with C++ code.
Python code to port:

```python
{python}
```
"""

In [8]:
def messages_for(python):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(python)}
    ]
 

In [9]:
def write_output(cpp):
    with open("main.cpp", "w") as f:
        f.write(cpp)

In [10]:
def port(model, python):
    client = clients[model]
    reasoning_effort = "high" if 'gpt' in model else None
    response = client.chat.completions.create(model=model, messages=messages_for(python), reasoning_effort=reasoning_effort)
    reply = response.choices[0].message.content
    reply = reply.replace('```cpp','').replace('```','')
    write_output(reply)
    return reply

In [11]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(200_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [12]:
def run_python(code):
    globals_dict = {"__builtins__": __builtins__}

    buffer = io.StringIO()
    old_stdout = sys.stdout
    sys.stdout = buffer

    try:
        exec(code, globals_dict)
        output = buffer.getvalue()
    except Exception as e:
        output = f"Error: {e}"
    finally:
        sys.stdout = old_stdout

    return output

In [14]:
def compile_and_run():
    try:
        subprocess.run(compile_command, check=True, text=True, capture_output=True)
        print(subprocess.run(run_command, check=True, text=True, capture_output=True).stdout)
        print(subprocess.run(run_command, check=True, text=True, capture_output=True).stdout)
        print(subprocess.run(run_command, check=True, text=True, capture_output=True).stdout)
    except subprocess.CalledProcessError as e:
        print(f"An error occurred:\n{e.stderr}")

In [15]:
with gr.Blocks() as ui:
    with gr.Row():
        python = gr.Textbox(label="Python code:", lines=28, value=pi)
        cpp = gr.Textbox(label="C++ code:", lines=28)
    with gr.Row():
        model = gr.Dropdown(models, label="Select model", value=models[0])
        convert = gr.Button("Convert code")

    convert.click(port, inputs=[model, python], outputs=[cpp])

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7883
* To create a public link, set `share=True` in `launch()`.
